# DEMO CRAWL DATA TỪ TRANG WEB nhatot.com
## 1. Mô tả quy trình thực hiện 
- Chương trình sẽ tự động truy cập vào trang web nhatot.com thông qua Webdriver
- Sau đó cũng sẽ tự động tắt đi các thành phần giao diện không cần thiết và có thể cản trở đến quy trình thu thập dữ liệu như thông báo, quảng cáo, gợi ý,...
- Chương trình sẽ được lặp qua từng trang sau đó sử dụng thư viện BeautifulSoup4 để phân tích nguồn trang và trích xuất dữ liệu mong muốn 
- Cuối cùng là đóng gói dữ liệu lại bằng file .csv và kết thúc driver
---
## 2. Thư viện sử dụng trong chương trình
- selenium: Tự động hóa quy trình thu thập dữ liệu
  - webdriver
  - By
  - NoSuchElementException
- BeautifulSoup: Phân tích và thu thập dữ liệu từ nguồn trang web
- sleep: Làm chậm chương trình nhằm vượt qua các cơ chế chống bot
- datetime: Xử lý thời gian thu thập dữ liệu
- pandas: Lưu trữ dữ liệu và chuyển đổi sang file .csv
---

In [ ]:
# Import các thư viện cần thiết
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

print("-   Finish importing modules   -")

## 3. Khởi tạo trình duyệt và truy cập trang web mục tiêu

In [ ]:
# Mở trình duyệt và truy cập vào trang web
driver = webdriver.Chrome()
url = 'https://www.nhatot.com/mua-ban-bat-dong-san-ha-noi'
driver.get(url)
print("-   Finish opening browser and accessing website   -")
sleep(3)
# Đợi cho trang web tải xong nội dung

In [ ]:
# Click 2 lần vào màn hình để tắt đi quảng cáo của trang web
sleep(5)
webdriver.ActionChains(driver).double_click().perform()

In [ ]:
# Tắt thông báo đăng nhập
sleep(3)
notification = driver.find_element(By.CLASS_NAME, 'close-btn')
notification.click()

---

## 4. Thu thập dữ liệu
### 4.1. Xây dựng hàm thu thập dữ liệu trên 1 trang
- Thông tin của mỗi bất động sản được chia làm các block trong thẻ li với itemprop = 'itemListElement'. Tuy nhiên ngoài các block chứa thông tin về bất động sản, còn có các item như bộc lọc của trang web,... chính vì thế ta cần lọc ra những block thực sự

In [ ]:
# Khoi tạo danh sách để lưu trữ dữ liệu và số trang để test
raw_data = []
number_of_pages = 2


In [ ]:
# Xây dựng hàm lấy dữ liệu nhà đất từ một trang
def get_house_data(driver, page_number):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Tìm tất cả các blocks chứa bài đăng nhà đất
    house_blocks = soup.find_all('li', itemprop = 'itemListElement')
    print(f"Tổng số bài đăng tìm thấy ở trang {page_number+1}: {len(house_blocks)}")
    
    results = []
    
    for i,block in enumerate(house_blocks):
        # Lấy thẻ a chứa link chính
        link_tag = block.select_one('a[itemprop="item"]')
        
        # Kiểm tra tính hợp lệ
        if not link_tag or not link_tag.has_attr('href'):
            continue
        raw_href = link_tag['href'].strip()
        
        # Lọc bỏ các href không phải bài đăng bất đọng sản
        if not raw_href.startswith('/mua-ban'): 
            continue
        # Ghép url chuẩn
        url = 'https://www.nhatot.com' + raw_href
        
        # 2. Lấy tt loại hình nhà đất, số pòng ngủ, hướng nhà
        type_tag = block.select_one('span.bwq0cbs.tle2ik0')
        property_type = type_tag.text.strip() if type_tag else None
        
        # 3. Lấy tt giá tổng, giá/m2, diện tích(nằm trên cùng 1 dòng)
        price_tags = block.select('span.bfe6oav')
        total_price = price_tags[0].text.strip() if len(price_tags) > 0 else None
        price_per_m2 = price_tags[1].text.strip() if len(price_tags) > 1 else None
        area = price_tags[2].text.strip() if len(price_tags) > 2 else None
        
        # 4. Lấy tt địa chỉ(quận) + thời gian thu thập
        location_tag = block.select_one('span.c1u6gyxh.tx5yyjc')
        location = location_tag.text.strip() if location_tag else None
        time = datetime.now().strftime('%Y-%m-%d')
        
        # Lưu bản ghi vào results
        results.append({
            'url': url,
            'property_type': property_type,
            'total_price': total_price,
            'price_per_m2': price_per_m2,
            'area': area,
            'location': location,
            'date': time
        })
        
    # Trả lại kết quả cuối
    return results 

### 4.2. Lặp qua các trang 
- nhatot.com có cơ chế chống bot đó là không cho phép các driver truy cập các đường link khác, đồng nghĩa ta sẽ không được chuyển sang trang tiếp theo bằng đường link có đuôi page=x
- Ta sẽ bắt buộc phải dùng selenium để tương tác với các button như người thật
- Qua quá trình kiểm tra có thể thấy nhatot.com có cơ chế đó là chỉ duy trì các bài đăng trong vòng 3 tuần sẽ được coi là "các tin mới hoặc tin ưu tiên" và những bài đăng quá hạn này sẽ không được hiển thị cho người dùng 
- Chính vì vậy mà trang web chỉ hiển thị tối đa 500 trang với mỗi trang chứa khoảng 23 - 25 bài đăng(không tính đến các tin quảng cáo)
- Ở trang 500 thì nút chuyển trang(next) sẽ được đổi tên và "disable" không cho phép người dùng thực hiện click

In [ ]:
# Lặp qua các trang bằng Selenium để thu thập dữ liệu
for page in range(number_of_pages):
    sleep(3)
    raw_data.extend(get_house_data(driver,page))
    # Cuộn đến khi gặp thẻ footer để đảm bảo không che đi thanh chuyển trang 
    while True:
        driver.execute_script("window.scrollBy(0, 500);")
        sleep(0.5)
        try:
            # Định vị đến vị trí để thanh chuyển trang hiện ra
            paging_div = driver.find_element(By.CLASS_NAME, "PtyCategoryDescription_contentWrap__uxtat")
            if paging_div.is_displayed():
                break
        except NoSuchElementException:
            continue
    sleep(2)
    try:
        # Kiểm tra xem có nút "next" còn hoạt động không
        next_icon = driver.find_element(By.CLASS_NAME, 'Paging_rightIconDisable__G58AE')
        print(f"Trang {page+1} là trang cuối cùng. Dừng vòng lặp.")
        break  # Đã đến trang cuối → không bấm next nữa
    except NoSuchElementException:
        try:
            # Nếu không phải disabled → vẫn còn trang sau → bấm next
            next_button = driver.find_element(By.CLASS_NAME, 'Paging_rightIcon__3p8MS')
            next_button.click()
            print(f"-   Hoàn thành thu thập dữ liệu ở trang {page+1}   -")
        except NoSuchElementException:
            print(f"Không tìm thấy nút next ở trang {page+1}. Dừng lại.")
            break

print(f"Số bài đăng hợp lệ: {len(raw_data)}")       

In [ ]:
print(raw_data[1])

### 4.3. Lưu trữ dữ liệu
- Dữ liệu sẽ được chuyển đổi sang DataFrame và lưu dưới dạng .csv

In [ ]:
df = pd.DataFrame(raw_data)
save_path = 'F:\\Data Analyst\\Viet Nam Real Estate Crawler\\bat_dong_san.csv'
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print("-   Finish saving data to CSV file   -")